<a href="https://colab.research.google.com/github/ChaithanyaSaiB/UMBC-DATA606-Capstone/blob/main/notebooks/Dataset_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Importing necessary libraries
import pandas as pd
import re
import os
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
# Import the tqdm library for progress bar
from tqdm import tqdm

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Load CNN health data
cnnhealth = pd.read_csv("/content/cnnhealth.txt", sep=r"(?<!\s)[|](?!\s)", names=['ID', 'DateTime', 'Article Title and Link'], engine="python")
cnnhealth.head()

,ID,DateTime,Article Title and Link
0,576880531301801984,Sat Mar 14 23:00:11 +0000 2015,An abundance of online info can turn us into e...
1,576820122666471424,Sat Mar 14 19:00:08 +0000 2015,A plant-based diet that incorporates fish may ...
2,576744652717461504,Sat Mar 14 14:00:15 +0000 2015,It doesn't take much to damage your hearing at...
3,576736754436304896,Sat Mar 14 13:28:52 +0000 2015,RT @CNN: Forever young? Discover this island’s...
4,576736614766010368,Sat Mar 14 13:28:18 +0000 2015,RT @CNN: Is post-traumatic stress disorder in ...


In [ ]:
# Extract all URLs from article titles and links
cnnhealth['URL'] = [re.findall(r'https?://\S+', text) for text in cnnhealth['Article Title and Link']]

# Drop rows with multiple or no URLs
drop_indices = cnnhealth['Article Title and Link'][cnnhealth['URL'].apply(len) != 1].index
cnnhealth.drop(drop_indices, inplace=True)

# Extract CNN URL for articles
cnnhealth['URL'] = [url[0] for url in cnnhealth['URL']]

In [ ]:
# Function to fetch content from a URL
def fetch_content(url):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            return response.text
        else:
            return None
    except Exception as e:
        print("Error fetching content:", e)
        return None

# Function to extract body content from HTML using BeautifulSoup
def extract_body(html_content):
    if html_content:
        soup = BeautifulSoup(html_content, 'html.parser')
        article_body = soup.findAll('p', class_='paragraph')
        if article_body:
            return article_body
        else:
            return None
    else:
        return None

# Apply web scraping to extract content and store it in a new column
def extract_content_with_progress(url_list):
    content_list = []
    for url in tqdm(url_list, desc='Extracting content'):
        content = extract_body(fetch_content(url))
        content_list.append(content)
    return content_list

# Apply web scraping to extract content and store it in a new column
cnnhealth['Content'] = extract_content_with_progress(cnnhealth['URL'])

Extracting content:   0%|          | 15/3299 [00:14<40:11,  1.36it/s]/usr/lib/python3.10/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)
Extracting content:   3%|▎         | 113/3299 [01:15<27:07,  1.96it/s]

Error fetching content: Failed to parse: https://t…


Extracting content:   6%|▌         | 200/3299 [02:04<41:49,  1.23it/s]

Error fetching content: Failed to parse: http://t.c…


Extracting content:   7%|▋         | 246/3299 [02:29<24:43,  2.06it/s]

...

Error fetching content: HTTPConnectionPool(host='t.', port=80): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7b450f139e40>: Failed to resolve 't' ([Errno -2] Name or service not known)"))


Extracting content:  81%|████████  | 2662/3299 [26:34<00:13, 47.87it/s]

Error fetching content: HTTPConnectionPool(host='t.c', port=80): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7b450f13b610>: Failed to resolve 't.c' ([Errno -2] Name or service not known)"))


Extracting content: 100%|██████████| 3299/3299 [26:55<00:00,  2.04it/s]

Error fetching content: HTTPConnectionPool(host='on.cnn.com', port=80): Max retries exceeded with url: /SO1PxT (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7b450dbd85b0>, 'Connection to on.cnn.com timed out. (connect timeout=10)'))


In [ ]:
# Drop rows with missing content
cnnhealth.dropna(subset=['Content'], inplace=True)

# Combine content paragraphs into a single string
cnnhealth['Content'] = [" ".join([p_tag.text.strip() for p_tag in content]) for content in cnnhealth['Content']]

In [ ]:
# Clean the content data
def clean_text(text):
    text = re.sub(r'-', ' ', text)  # Substitute hyphens with empty spaces
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphabetic characters
    text = text.lower()  # Convert text to lowercase
    return text

cnnhealth['Content'] = cnnhealth['Content'].apply(clean_text)

In [ ]:
# Convert to wordnet tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None  # Use default POS for lemmatization

In [ ]:
# Tokenize and preprocess text data
def preprocess_text(text):
    tokens = word_tokenize(text)  # Tokenize text
    lemmatizer = WordNetLemmatizer()  # Initialize lemmatizer
    pos_tags = nltk.pos_tag(tokens)  # Get part-of-speech tags
    for i, (token, tag) in enumerate(pos_tags):
        pos = get_wordnet_pos(tag)  # Convert NLTK POS tags to WordNet POS tags
        if pos:
            tokens[i] = lemmatizer.lemmatize(token, pos=pos)  # Lemmatize tokens
        else:
            tokens[i] = lemmatizer.lemmatize(token)  # Use default POS for lemmatization
    stop_words = set(stopwords.words('english'))  # Get stopwords
    custom_stopwords = [    # Custom stopwords
    "patient", "doctor", "say", "year", "state", "day", "need", "come", "well",
    "make", "think", "know", "go", "use", "one", "like", "people", "may",
    "many", "still", "even", "two", "way", "good", "much", "back", "new",
    "time", "first", "really",
    "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n",
    "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"
    ]
    tokens = [token for token in tokens if token not in stop_words and token not in custom_stopwords]  # Remove stopwords
    return ' '.join(tokens)

cnnhealth['Content'] = cnnhealth['Content'].apply(preprocess_text)

In [ ]:
# Save preprocessed data to CSV
cnnhealth.to_csv('cnnhealth_preprocessed.csv', index=False)

In [3]:
# Define the directory path containing the .txt files
directory = '/content'

# Initialize an empty list to store DataFrames
dfs = []

# Iterate over all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        # Read the .txt file into a DataFrame
        df_name = os.path.splitext(filename)[0]  # Extract filename without extension
        try:
            df = pd.read_csv(os.path.join(directory, filename),
                             sep=r"(?<!\s)[|](?!\s)",
                             names=['ID', 'DateTime', 'Article Title and Link'],
                             engine="python")
            # Add the DataFrame to the list
            dfs.append(df)
            print(f"DataFrame '{df_name}' loaded with {len(df)} rows.")
        except UnicodeDecodeError:
            print(f"Error reading {filename}: UnicodeDecodeError. Skipping this file.")

# Concatenate all DataFrames in the list into one DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Display the combined DataFrame
print("Combined DataFrame:")
print(combined_df)

DataFrame 'everydayhealth' loaded with 3239 rows.
DataFrame 'cnnhealth' loaded with 4061 rows.
DataFrame 'latimeshealth' loaded with 4171 rows.
DataFrame 'nprhealth' loaded with 4837 rows.
DataFrame 'cbchealth' loaded with 3741 rows.
Combined DataFrame:
                       ID                        DateTime  \
0      304596701757464576  Thu Feb 21 14:21:27 +0000 2013   
1      304595191329853441  Thu Feb 21 14:15:27 +0000 2013   
2      304587659018371072  Thu Feb 21 13:45:31 +0000 2013   
3      304580073380524032  Thu Feb 21 13:15:22 +0000 2013   
4      304572560270573569  Thu Feb 21 12:45:31 +0000 2013   
...                   ...                             ...   
20044  324528904306057216  Wed Apr 17 14:24:54 +0000 2013   
20045  324503137597665282  Wed Apr 17 12:42:31 +0000 2013   
20046  324261559025160193  Tue Apr 16 20:42:34 +0000 2013   
20047  324251178919657474  Tue Apr 16 20:01:20 +0000 2013   
20048  324205113348616192  Tue Apr 16 16:58:17 +0000 2013   

             

In [4]:
# Extract all URLs from article titles and links in combined_df
combined_df['URL'] = [re.findall(r'https?://\S+', text) for text in combined_df['Article Title and Link']]

# Drop rows with multiple or no URLs
drop_indices = combined_df['Article Title and Link'][combined_df['URL'].apply(len) != 1].index
combined_df.drop(drop_indices, inplace=True)

# Extract URL for articles
combined_df['URL'] = [url[0] for url in combined_df['URL']]


In [5]:
!pip install fake-useragent

In [6]:
split_1 = combined_df.iloc[:5000].copy()
split_2 = combined_df.iloc[5000:10000].copy()
split_3 = combined_df.iloc[10000:15000].copy()
split_4 = combined_df.iloc[15000:20000].copy()
split_5 = combined_df.iloc[20000:].copy()

In [7]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from urllib.parse import urlparse

# Create a UserAgent instance
ua = UserAgent()

# Function to fetch content from a URL
def fetch_content(url):
    try:
        # Define user-agent header using a random user agent
        headers = {'User-Agent': ua.random}

        # Create a session to handle cookies and maintain connection state
        with requests.Session() as session:
            response = session.get(url, headers=headers, timeout=4, allow_redirects=True)
            response.raise_for_status()  # Raise an error for 4xx and 5xx status codes
            final_url = response.url  # Get the final URL after following redirects
            website_name = get_website_name(final_url)
            return (response.text, website_name)
    except Exception as e:
        print("Error fetching content:", e)
        return (None, None)

# Function to extract website name from URL
def get_website_name(url):
    parsed_url = urlparse(url)
    return parsed_url.netloc

# Function to extract body content from HTML using BeautifulSoup
def extract_body(html_content_with_website_name):
    html_content, website_name = html_content_with_website_name
    if html_content:
        soup = BeautifulSoup(html_content, 'html.parser')
        if website_name == 'www.cnn.com':
          article_body = soup.findAll('p', class_='paragraph')
        elif website_name == 'www.health.usnews.com':
          article_body = soup.findAll('p')
        elif website_name == 'www.latimes.com':
          article_body = soup.findAll('p')
        elif website_name == 'www.npr.org':
          article_body = soup.find('div', id='storytext').find_all('p')
        elif website_name == 'www.nytimes.com':
          article_body = soup.findAll('p', class_='story-body-text')
        else:
          article_body = None

        if article_body:
            return article_body
        else:
            return None
    else:
        return None

# Apply web scraping to extract content and store it in a new column
def extract_content_with_progress(url_list):
    content_list = []
    for url in tqdm(url_list, desc='Extracting content'):
        content = extract_body(fetch_content(url))
        content_list.append(content)
    return content_list

# Apply web scraping to extract content and store it in a new column
#combined_df['Content'] = extract_content_with_progress(combined_df['URL'])

In [8]:
split_1['Content'] = extract_content_with_progress(split_1['URL'])
split_1.to_csv('split_1_preprocessed.csv', index=False)

Extracting content:   0%|          | 13/5000 [00:31<2:50:32,  2.05s/it]

Error fetching content: ('Connection aborted.', HTTPException('got more than 100 headers'))


Extracting content:   1%|          | 44/5000 [01:28<2:07:50,  1.55s/it]

Error fetching content: 410 Client Error: Gone for url: https://www.everydayhealth.com/healthy-living-pictures/consumer-electronics-show-health.aspx#/slide-1


Extracting content:   1%|          | 49/5000 [01:35<1:45:33,  1.28s/it]

...

Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/y9lMVA7
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/AlCB33I
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/r6pZXEw
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/2QOrgxd
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/xsH9wmh
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/puBCFE8
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/z86uglb
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/NCD2bIo
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/DLNXjom
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/Y4GQsE9


Extracting content: 100%|█████████▉| 4983/5000 [1:41:07<00:02,  6.91it/s]

Error fetching content: HTTPSConnectionPool(host='makingcancerhistory.com', port=443): Max retries exceeded with url: /WhyMDA/Myths.html (Caused by SSLError(SSLError(1, '[SSL: WRONG_SIGNATURE_TYPE] wrong signature type (_ssl.c:1007)')))
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/keesFm8
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/dntM3I6
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/heZEauq
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/zAY0jdL
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/1ypDGAa
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/j3Nr7Uv
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/HT9EwCH
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/CeyGFdk
Error fetch

Extracting content: 100%|██████████| 5000/5000 [1:41:07<00:00,  1.21s/it]

Error fetching content: 404 Client Error: Not Found for url: https://t.co/SV
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/0o4Jsnh


In [ ]:
split_2['Content'] = extract_content_with_progress(split_2['URL'])
split_2.to_csv('split_2_preprocessed.csv', index=False)

In [ ]:
split_3['Content'] = extract_content_with_progress(split_3['URL'])
split_3.to_csv('split_3_preprocessed.csv', index=False)

In [ ]:
split_4['Content'] = extract_content_with_progress(split_4['URL'])
split_4.to_csv('split_4_preprocessed.csv', index=False)

In [ ]:
split_5['Content'] = extract_content_with_progress(split_5['URL'])
split_5.to_csv('split_5_preprocessed.csv', index=False)

In [ ]:
# Drop rows with missing content
combined_df.dropna(subset=['Content'], inplace=True)

# Combine content paragraphs into a single string
combined_df['Content'] = [" ".join([p_tag.text.strip() for p_tag in content]) for content in combined_df['Content']]

In [ ]:
# Clean the content data
def clean_text(text):
    text = re.sub(r'-', ' ', text)  # Substitute hyphens with empty spaces
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphabetic characters
    text = text.lower()  # Convert text to lowercase
    return text

combined_df['Content'] = combined_df['Content'].apply(clean_text)

In [ ]:
# Convert to wordnet tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None  # Use default POS for lemmatization

In [ ]:
# Tokenize and preprocess text data
def preprocess_text(text):
    tokens = word_tokenize(text)  # Tokenize text
    lemmatizer = WordNetLemmatizer()  # Initialize lemmatizer
    pos_tags = nltk.pos_tag(tokens)  # Get part-of-speech tags
    for i, (token, tag) in enumerate(pos_tags):
        pos = get_wordnet_pos(tag)  # Convert NLTK POS tags to WordNet POS tags
        if pos:
            tokens[i] = lemmatizer.lemmatize(token, pos=pos)  # Lemmatize tokens
        else:
            tokens[i] = lemmatizer.lemmatize(token)  # Use default POS for lemmatization
    stop_words = set(stopwords.words('english'))  # Get stopwords
    custom_stopwords = [    # Custom stopwords
    "patient", "doctor", "say", "year", "state", "day", "need", "come", "well",
    "make", "think", "know", "go", "use", "one", "like", "people", "may",
    "many", "still", "even", "two", "way", "good", "much", "back", "new",
    "time", "first", "really",
    "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n",
    "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"
    ]
    tokens = [token for token in tokens if token not in stop_words and token not in custom_stopwords]  # Remove stopwords
    return ' '.join(tokens)

combined_df['Content'] = combined_df['Content'].apply(preprocess_text)

In [ ]:
# Save preprocessed data to CSV
combined_df.to_csv('combined_preprocessed.csv', index=False)